# Skin Disease Classification using EfficientNetB0

This notebook implements skin disease classification using the EfficientNetB0 architecture.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

## 1. Data Loading and Preprocessing

In [ ]:
# Load metadata
metadata = pd.read_csv('../data/skin_disease_images/HAM10000_metadata.csv')

# Create image data generators with advanced augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

## 2. Model Architecture

In [ ]:
def build_efficientnet_model(num_classes):
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze the base model initially
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

## 3. Training with Advanced Techniques

In [ ]:
# Model compilation with learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9
)

model = build_efficientnet_model(num_classes=7)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

# Advanced callbacks
checkpoint = ModelCheckpoint(
    'best_model_efficientnet.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

## 4. Advanced Evaluation and Visualization

In [ ]:
def plot_roc_curves(y_true, y_pred_proba, classes):
    plt.figure(figsize=(10, 8))
    for i in range(len(classes)):
        fpr, tpr, _ = roc_curve(y_true[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{classes[i]} (AUC = {roc_auc:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Each Class')
    plt.legend()
    plt.show()

def visualize_feature_maps(model, image, layer_name):
    layer_output = model.get_layer(layer_name).output
    activation_model = tf.keras.Model(inputs=model.input, outputs=layer_output)
    activation = activation_model.predict(image)
    
    plt.figure(figsize=(15, 8))
    for i in range(min(16, activation.shape[-1])):
        plt.subplot(4, 4, i + 1)
        plt.imshow(activation[0, :, :, i], cmap='viridis')
        plt.axis('off')
    plt.suptitle(f'Feature Maps from {layer_name}')
    plt.show()

In [ ]:
# Create model saving directory
model_save_dir = os.path.join('..', 'models', 'efficientnet')
os.makedirs(model_save_dir, exist_ok=True)

# Update checkpoint callback
checkpoint = ModelCheckpoint(
    os.path.join(model_save_dir, 'best_model_efficientnet.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# Add model saving after training
model.save(os.path.join(model_save_dir, 'final_model_efficientnet.h5'))